In [ ]:
# notebook used to link imaging reports to manual annotations done according to PRISSMM.

In [1]:
import numpy as np
import pandas as pd

In [2]:
def find_metsite(dataset, mincode, maxcode):
    temp = np.where(((dataset['image_casite1'] >= mincode) & (dataset['image_casite1'] <= maxcode)) |
                    ((dataset['image_casite2'] >= mincode) & (dataset['image_casite2'] <= maxcode)) |
                    ((dataset['image_casite3'] >= mincode) & (dataset['image_casite3'] <= maxcode)) |
                    ((dataset['image_casite4'] >= mincode) & (dataset['image_casite4'] <= maxcode)) |
                    ((dataset['image_casite5'] >= mincode) & (dataset['image_casite5'] <= maxcode)) |
                    ((dataset['image_casite6'] >= mincode) & (dataset['image_casite6'] <= maxcode)) |
                    ((dataset['image_casite7'] >= mincode) & (dataset['image_casite7'] <= maxcode)) |
                    ((dataset['image_casite8'] >= mincode) & (dataset['image_casite8'] <= maxcode)) |
                    ((dataset['image_casite9'] >= mincode) & (dataset['image_casite9'] <= maxcode)) |
                    ((dataset['image_casite10'] >= mincode) & (dataset['image_casite10'] <= maxcode)) |
                    ((dataset['image_casite11'] >= mincode) & (dataset['image_casite11'] <= maxcode)) |
                    ((dataset['image_casite12'] >= mincode) & (dataset['image_casite12'] <= maxcode)) |
                    ((dataset['image_casite13'] >= mincode) & (dataset['image_casite13'] <= maxcode)) |
                    ((dataset['image_casite14'] >= mincode) & (dataset['image_casite14'] <= maxcode)) |
                    ((dataset['image_casite15'] >= mincode) & (dataset['image_casite15'] <= maxcode)),
                    1, 0)
    return temp
    #return np.where(np.isnan(temp), 0, temp)

In [3]:
def parse_imaging_file(dataset):
    newset = dataset.copy()
    newset = newset[newset.redcap_repeat_instrument == 'prissmm_imaging']
    newset = newset[newset.prissmm_imaging_complete == 2]
    newset['brain_met'] = find_metsite(newset, 700, 719)
    newset['bone_met'] = find_metsite(newset, 400, 419)
    newset['adrenal_met'] = find_metsite(newset, 740, 749)
    newset['liver_met'] = find_metsite(newset, 220, 220)
    newset['lung_met'] = find_metsite(newset, 340, 349)
    newset['node_met'] = find_metsite(newset, 770, 779)
    newset['peritoneal_met'] = find_metsite(newset, 481, 482)
    newset['ascites'] = find_metsite(newset, 901, 901)
    newset['peritoneal_met'] = np.where((newset.peritoneal_met==1) | (newset.ascites==1), 1, 0)
    newset['any_cancer'] = np.where(newset['image_ca'] == 1, 1, 0)
    newset['response'] = np.where(newset['image_overall'] == 1, 1, 0)
    newset['progression'] = np.where(newset['image_overall'] == 4, 1, 0)
    newset['class_status'] = np.where(newset['any_cancer'] == 0, 0, newset['image_overall'])
    newset = newset.rename(columns={'image_scansite___1':'head_imaged',
                           'image_scansite___2':'spine_imaged',
                           'image_scansite___3':'neck_imaged',
                           'image_scansite___4':'chest_imaged',
                           'image_scansite___5':'abdomen_imaged',
                           'image_scansite___6':'pelvis_imaged',
                           'image_scansite___7':'extremity_imaged',
                           'image_scansite___8':'whole_body_imaged'})
    newset['head_imaged'] = np.where(newset['whole_body_imaged']==1, 1, newset['head_imaged'])
    newset['spine_imaged'] = np.where(newset['whole_body_imaged']==1, 1, newset['spine_imaged'])
    newset['neck_imaged'] = np.where(newset['whole_body_imaged']==1, 1, newset['neck_imaged'])
    newset['chest_imaged'] = np.where(newset['whole_body_imaged']==1, 1, newset['chest_imaged'])
    newset['abdomen_imaged'] = np.where(newset['whole_body_imaged']==1, 1, newset['abdomen_imaged'])
    newset['pelvis_imaged'] = np.where(newset['whole_body_imaged']==1, 1, newset['pelvis_imaged'])
    newset['extremity_imaged'] = np.where(newset['whole_body_imaged']==1, 1, newset['extremity_imaged'])
    newset['date'] = pd.to_datetime(newset.image_scan_dt)

    return newset[['record_id','date','image_scan_type','any_cancer','progression','response','class_status','brain_met','bone_met','adrenal_met','liver_met','lung_met','node_met','peritoneal_met','head_imaged','neck_imaged','spine_imaged','chest_imaged','abdomen_imaged','pelvis_imaged','extremity_imaged','whole_body_imaged']]

In [19]:
# pull text
all_imaging = pd.read_csv('/mnt/d/Dropbox (Partners Healthcare)/profile_3-2023/derived_data/all_imaging_for_prissmm.csv')
all_imaging = all_imaging[~all_imaging.scan_type.str.contains('NO INTERPRETATION')]
all_imaging['date'] = pd.to_datetime(all_imaging.date)

all_imaging['image_scan_type'] = np.where(all_imaging.scan_type.str[:2] == 'CT', 1, 99)
all_imaging['image_scan_type'] = np.where(all_imaging.scan_type.str[:3] == 'MRI', 3, all_imaging['image_scan_type'])
all_imaging['image_scan_type'] = np.where(all_imaging.scan_type.str[:6] == 'NM PET', 5, all_imaging['image_scan_type'])
all_imaging['image_scan_type'] = np.where(all_imaging.scan_type.str[:7] == 'NM BONE', 7, all_imaging['image_scan_type'])
all_imaging['image_scan_type'] = np.where(all_imaging.scan_type.str[:8] == 'BI MAMMO', 11, all_imaging['image_scan_type'])

all_imaging = all_imaging[all_imaging.image_scan_type != 99]

all_imaging['head_imaged'] = np.where(all_imaging.scan_type.str.contains('BRAIN|HEAD') | all_imaging.image_scan_type.isin([5, 7]), 1, 0)
all_imaging['neck_imaged'] = np.where(all_imaging.scan_type.str.contains('NECK') | all_imaging.image_scan_type.isin([5,7]), 1, 0)
all_imaging['spine_imaged'] = np.where(all_imaging.scan_type.str.contains('SPINE') | all_imaging.image_scan_type.isin([5,7]), 1, 0)
all_imaging['chest_imaged'] = np.where(all_imaging.scan_type.str.contains('CHEST') | all_imaging.image_scan_type.isin([5,7,11]), 1, 0)
all_imaging['abdomen_imaged'] = np.where(all_imaging.scan_type.str.contains('ABDOMEN') | all_imaging.image_scan_type.isin([5,7]), 1, 0)
all_imaging['pelvis_imaged'] = np.where(all_imaging.scan_type.str.contains('PELVIS') | all_imaging.image_scan_type.isin([5,7]), 1, 0)




In [20]:
all_imaging.image_scan_type.value_counts()

1     379013
3     139375
5      39321
11     38183
7      13743
Name: image_scan_type, dtype: int64

In [21]:
all_imaging.primary_cancer_diagnosis.value_counts()[0:25]

Lung Adenocarcinoma                                            85525
Colon Adenocarcinoma                                           36899
Breast Invasive Ductal Carcinoma                               34237
Invasive Breast Carcinoma                                      32744
High-Grade Serous Ovarian Cancer                               22695
Melanoma                                                       19960
Pancreatic Adenocarcinoma                                      18813
Prostate Adenocarcinoma                                        15209
Glioblastoma                                                   14798
Bladder Urothelial Carcinoma                                   13763
Lung Squamous Cell Carcinoma                                    9725
Renal Clear Cell Carcinoma                                      9438
Non-Small Cell Lung Cancer                                      8864
Rectal Adenocarcinoma                                           8604
Uterine Endometrioid Carcinoma    

In [22]:
all_imaging.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 609635 entries, 0 to 776301
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   Unnamed: 0                609635 non-null  int64         
 1   dfci_mrn                  609635 non-null  int64         
 2   date                      609635 non-null  datetime64[ns]
 3   text                      609635 non-null  object        
 4   impression_text           436265 non-null  object        
 5   scan_type                 609635 non-null  object        
 6   patient_id                609635 non-null  int64         
 7   hybrid_death_ind          609635 non-null  object        
 8   hybrid_death_dt           308914 non-null  object        
 9   primary_cancer_diagnosis  609635 non-null  object        
 10  genomics_date             609635 non-null  object        
 11  after_profile             609635 non-null  int64         
 12  sp

In [23]:
# different imaging report types
prefix = '/mnt/d/Dropbox (Partners Healthcare)/'
dataset_list = []
for cancer_type in ['nsclc_phase2_existing','crc','breast','pancreas','bladder','prostate']:
    pts = pd.read_csv(prefix+'profile_3-2023/redcaps/' + cancer_type + '_ptchars.csv', low_memory=False)[['record_id','internal_mrn']]
    pts = pts.sort_values(by=['record_id','internal_mrn']).reset_index(drop=True)
    pts = pts.groupby('internal_mrn').first().reset_index(drop=False).rename(columns={'internal_mrn':'dfci_mrn'})
    
    # for prostate patients only take BPC cases? that would be inconsistent...
    
    imaging = pd.read_csv(prefix+'profile_3-2023/redcaps/' + cancer_type + '_imaging.csv', low_memory=False)
    imaging = parse_imaging_file(imaging)
    imaging = pd.merge(pts, imaging, on='record_id').drop(columns='record_id')
    imaging['cancer_type'] = cancer_type
    temp = pd.merge(imaging, all_imaging, on=['dfci_mrn','image_scan_type','date','head_imaged','neck_imaged','spine_imaged','chest_imaged','abdomen_imaged','pelvis_imaged'])

    dataset_list.append(temp)

In [24]:
imaging.any_cancer.value_counts()

1    6089
0    3830
Name: any_cancer, dtype: int64

In [25]:
imaging.class_status.isnull().value_counts()

False    9919
Name: class_status, dtype: int64

In [26]:
imaging.class_status.value_counts()

0.0    3830
4.0    2361
5.0    1529
2.0    1510
1.0     419
3.0     270
Name: class_status, dtype: int64

In [27]:
# now do RCC
pts = pd.read_csv(prefix+'profile_3-2023/redcaps/rcc_barkouny_ptchars.csv', low_memory=False)[['record_id','local_mrn']]
pts = pts.sort_values(by=['record_id','local_mrn']).reset_index(drop=True)
pts = pts.groupby('local_mrn').first().reset_index(drop=False).rename(columns={'local_mrn':'dfci_mrn'})
    
imaging = pd.read_csv(prefix+'profile_3-2023/redcaps/rcc_barkouny_imaging.csv', low_memory=False)
imaging = imaging[imaging.redcap_repeat_instrument == 'prissmm_imaging_meta_data']

imaging['image_scan_dt'] = np.where(imaging.image_scan_dt.isnull(), imaging.image_report_dt, imaging.image_scan_dt)

imaging['any_cancer'] = np.where(imaging['image_ca'] == 1, 1, 0)
imaging['response'] = np.where(imaging['image_overall'] == 1, 1, 0)
imaging['progression'] = np.where(imaging['image_overall'] == 4, 1, 0)
imaging['class_status'] = np.where(imaging['any_cancer'] == 0, 0, imaging['image_overall'])

imaging = imaging.rename(columns={'image_scansite___1':'head_imaged',
                       'image_scansite___2':'neck_imaged',
                       'image_scansite___3':'chest_imaged',
                       'image_scansite___4':'abdomen_imaged',
                       'image_scansite___5':'pelvis_imaged',
                       'image_scansite___6':'extremity_imaged',
                       'image_scansite___7':'whole_body_imaged'})
imaging['head_imaged'] = np.where(imaging['whole_body_imaged']==1, 1, imaging['head_imaged'])
imaging['neck_imaged'] = np.where(imaging['whole_body_imaged']==1, 1, imaging['neck_imaged'])
imaging['chest_imaged'] = np.where(imaging['whole_body_imaged']==1, 1, imaging['chest_imaged'])
imaging['abdomen_imaged'] = np.where(imaging['whole_body_imaged']==1, 1, imaging['abdomen_imaged'])
imaging['pelvis_imaged'] = np.where(imaging['whole_body_imaged']==1, 1, imaging['pelvis_imaged'])
imaging['extremity_imaged'] = np.where(imaging['whole_body_imaged']==1, 1, imaging['extremity_imaged'])
imaging['date'] = pd.to_datetime(imaging.image_scan_dt)



imaging['brain_met'] = imaging.image_site___30
imaging['bone_met'] = imaging.image_site___25
imaging['adrenal_met'] = imaging.image_site___1
imaging['liver_met'] = imaging.image_site___95
imaging['lung_met'] = imaging.image_site___100
imaging['node_met'] = np.where((imaging.image_site___105==1) | (imaging.image_site___110==1) | (imaging.image_site___115==1), 1, 0)
imaging['peritoneal_met'] = imaging.image_site___150

imaging = pd.merge(pts, imaging, on='record_id').reset_index()
imaging['cancer_type'] = 'rcc_barkouny'

imaging['spine_imaged'] = 99


imaging = imaging[['dfci_mrn','date','image_scan_type','any_cancer','progression','response','class_status','brain_met','bone_met','adrenal_met','liver_met','lung_met','node_met','peritoneal_met','head_imaged','neck_imaged','spine_imaged','cancer_type','chest_imaged','abdomen_imaged','pelvis_imaged','extremity_imaged','whole_body_imaged']]

temp = pd.merge(imaging, all_imaging.drop(columns='spine_imaged'), on=['dfci_mrn','image_scan_type','date','head_imaged','neck_imaged','chest_imaged','abdomen_imaged','pelvis_imaged'])
dataset_list.append(temp)

In [28]:
dataset_list[5].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5763 entries, 0 to 5762
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   dfci_mrn                  5763 non-null   int64         
 1   date                      5763 non-null   datetime64[ns]
 2   image_scan_type           5763 non-null   float64       
 3   any_cancer                5763 non-null   int64         
 4   progression               5763 non-null   int64         
 5   response                  5763 non-null   int64         
 6   class_status              5763 non-null   float64       
 7   brain_met                 5763 non-null   int64         
 8   bone_met                  5763 non-null   int64         
 9   adrenal_met               5763 non-null   int64         
 10  liver_met                 5763 non-null   int64         
 11  lung_met                  5763 non-null   int64         
 12  node_met            

In [29]:
final = pd.concat(dataset_list, axis=0)

In [30]:
final.cancer_type.value_counts()

nsclc_phase2_existing    10524
breast                    6888
crc                       6805
prostate                  5763
pancreas                  3598
bladder                   3498
rcc_barkouny              3254
Name: cancer_type, dtype: int64

In [31]:
final.image_scan_type.value_counts()

1.0     23359
3.0      8648
7.0      4127
5.0      3342
11.0      854
Name: image_scan_type, dtype: int64

In [33]:
final = final.groupby(['dfci_mrn','cancer_type','image_scan_type','date','head_imaged','neck_imaged','spine_imaged','chest_imaged','abdomen_imaged','pelvis_imaged']).first().reset_index(drop=False)

In [34]:
final.class_status.value_counts()

0.0    16801
4.0     8185
2.0     5600
5.0     3924
1.0     2352
3.0      661
Name: class_status, dtype: int64

In [35]:
# drop duplicate reports; the most common reason for this was a CT chest/abdomen/pelvis with identical interpretations entered for the chest and the abdomen/pelvis
final = final.drop_duplicates(subset='text').reset_index(drop=True)
print(final.info())
print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37452 entries, 0 to 37451
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   dfci_mrn                  37452 non-null  float64       
 1   cancer_type               37452 non-null  object        
 2   image_scan_type           37452 non-null  float64       
 3   date                      37452 non-null  datetime64[ns]
 4   head_imaged               37452 non-null  float64       
 5   neck_imaged               37452 non-null  float64       
 6   spine_imaged              37452 non-null  float64       
 7   chest_imaged              37452 non-null  float64       
 8   abdomen_imaged            37452 non-null  float64       
 9   pelvis_imaged             37452 non-null  float64       
 10  any_cancer                37452 non-null  int64         
 11  progression               37452 non-null  int64         
 12  response          

In [36]:
# drop 'reports' of outside scans, which just indicate that the scan was performed but do not contain useful text
final = final[~final['text'].str.contains("It has been imported")]
print(final.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37274 entries, 0 to 37451
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   dfci_mrn                  37274 non-null  float64       
 1   cancer_type               37274 non-null  object        
 2   image_scan_type           37274 non-null  float64       
 3   date                      37274 non-null  datetime64[ns]
 4   head_imaged               37274 non-null  float64       
 5   neck_imaged               37274 non-null  float64       
 6   spine_imaged              37274 non-null  float64       
 7   chest_imaged              37274 non-null  float64       
 8   abdomen_imaged            37274 non-null  float64       
 9   pelvis_imaged             37274 non-null  float64       
 10  any_cancer                37274 non-null  int64         
 11  progression               37274 non-null  int64         
 12  response          

In [37]:
final.cancer_type.value_counts()

nsclc_phase2_existing    9954
crc                      6478
breast                   6066
prostate                 5106
pancreas                 3423
bladder                  3261
rcc_barkouny             2986
Name: cancer_type, dtype: int64

In [38]:
final.groupby('dfci_mrn').first().cancer_type.value_counts()

crc                      629
nsclc_phase2_existing    614
prostate                 480
rcc_barkouny             412
pancreas                 402
breast                   368
bladder                  308
Name: cancer_type, dtype: int64

In [39]:
final.to_csv(prefix+'profile_3-2023/derived_data/labeled_imaging_prissmm.csv')

In [40]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37274 entries, 0 to 37451
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   dfci_mrn                  37274 non-null  float64       
 1   cancer_type               37274 non-null  object        
 2   image_scan_type           37274 non-null  float64       
 3   date                      37274 non-null  datetime64[ns]
 4   head_imaged               37274 non-null  float64       
 5   neck_imaged               37274 non-null  float64       
 6   spine_imaged              37274 non-null  float64       
 7   chest_imaged              37274 non-null  float64       
 8   abdomen_imaged            37274 non-null  float64       
 9   pelvis_imaged             37274 non-null  float64       
 10  any_cancer                37274 non-null  int64         
 11  progression               37274 non-null  int64         
 12  response          

In [41]:
profile_consents = pd.read_csv(prefix+'profile_3-2023/structured_data/REQ_KK71_105304_2_PROFILE_CONSENT.csv', low_memory=False)
profile_consents = profile_consents[~profile_consents.DRIVING_PROTOCOL_NBR.isnull()]
profile_consents = profile_consents[profile_consents.DATA_SHARING == 'Y']

In [62]:
final_consented = final[final.dfci_mrn.isin(profile_consents.DFCI_MRN)]

In [63]:
final_consented.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34358 entries, 0 to 36727
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   dfci_mrn                  34358 non-null  float64       
 1   cancer_type               34358 non-null  object        
 2   image_scan_type           34358 non-null  float64       
 3   date                      34358 non-null  datetime64[ns]
 4   head_imaged               34358 non-null  float64       
 5   neck_imaged               34358 non-null  float64       
 6   spine_imaged              34358 non-null  float64       
 7   chest_imaged              34358 non-null  float64       
 8   abdomen_imaged            34358 non-null  float64       
 9   pelvis_imaged             34358 non-null  float64       
 10  any_cancer                34358 non-null  int32         
 11  progression               34358 non-null  int32         
 12  response          

In [64]:
final_consented.to_csv(prefix+'profile_3-2023/derived_data/labeled_imaging_prissmm_profile_consented.csv')